## 코스피 지수 데이터

In [82]:
#https://github.com/sharebook-kr/pykrx          -> 한국 개발자가 만든 주식 정보 스크래핑 라이브러리
!pip install pykrx
from pykrx import stock
import numpy as np

In [83]:
df = stock.get_index_ohlcv_by_date("20200101","20200831","1001")
df = df[['종가']]
df.head()

코스피,종가
날짜,
2020-01-02,2175.17
2020-01-03,2176.46
2020-01-06,2155.07
2020-01-07,2175.54
2020-01-08,2151.31


## 이동 평균선 매매 <br>

가격이 이동평균선을 돌파하면 매수, 그렇지 않으면 매도

In [84]:
df['ma'] = df['종가'].rolling( 3 ).mean().shift(1) #rolling 그룹화해주는 함수 -> 그룹화하여 평균을 적용하라
df.head()

코스피,종가,ma
날짜,,
2020-01-02,2175.17,NaN
2020-01-03,2176.46,NaN
2020-01-06,2155.07,NaN
2020-01-07,2175.54,2168.900000
2020-01-08,2151.31,2169.023333


In [85]:
cond  = df['종가'] > df['ma']
df['status'] = np.where(cond,1,0)
df.iloc[-1,-1] = 0
df.head()

코스피,종가,ma,status
날짜,,,
2020-01-02,2175.17,NaN,0
2020-01-03,2176.46,NaN,0
2020-01-06,2155.07,NaN,0
2020-01-07,2175.54,2168.900000,1
2020-01-08,2151.31,2169.023333,0


In [86]:
## 0 -> 1 골든크로스 : 매수
## 1 -> 0 데드크로스 : 매도 
df['shift'] = df['status'].shift(1) #df.shift(1) 은 한칸 전에 있던 데이터를 뜻함
df.drop(columns = 'shift')

코스피,종가,ma,status
날짜,,,
2020-01-02,2175.17,NaN,0
2020-01-03,2176.46,NaN,0
2020-01-06,2155.07,NaN,0
2020-01-07,2175.54,2168.900000,1
2020-01-08,2151.31,2169.023333,0
...,...,...,...
2020-08-25,2366.73,2302.880000,1
2020-08-26,2369.32,2333.716667,1
2020-08-27,2344.45,2355.293333,0


In [87]:
매수조건 = (df['status'] == 1) & (df['status'].shift(1) != 1) #0->1
매도조건 = (df['status'] == 0) & (df['status'].shift(1) == 1) #1->0

In [88]:
df[매수조건].head()

코스피,종가,ma,status,shift
날짜,,,,
2020-01-07,2175.54,2168.900000,1,0.0
2020-01-09,2186.45,2160.640000,1,0.0
2020-01-22,2267.25,2250.966667,1,0.0
2020-02-04,2157.90,2128.630000,1,0.0
2020-02-11,2223.12,2213.653333,1,0.0


In [89]:
df[매도조건].head()

코스피,종가,ma,status,shift
날짜,,,,
2020-01-08,2151.31,2169.023333,0,1.0
2020-01-21,2239.69,2253.753333,0,1.0
2020-01-23,2246.13,2256.526667,0,1.0
2020-02-10,2201.07,2201.840000,0,1.0
2020-02-18,2208.88,2239.573333,0,1.0


In [90]:
수익률 = df.loc[매도조건,'종가'].reset_index(drop=True)/df.loc[매수조건,'종가'].reset_index(drop=True)
수익률 = 수익률 - 0.002
수익률.cumprod().iloc[-1]

1.2037220606995058

### 함수화 하기

In [91]:
def 단일_이평선_수익률(df,N):
    df = df[['종가']].copy()
    
    #3일 이평선 생성
    df['ma'] = df['종가'].rolling( N ).mean().shift(1)
    
    #종가가 이평선보다 높은 날에 모두 1, 그렇지 않은 날에 0을 추가 (상승 추세판단)
    cond  = df['종가'] > df['ma']
    df['status'] = np.where(cond,1,0)
    df.iloc[-1,-1] = 0
    
    #bool 조건식화 하기
    매수조건 = (df['status'] == 1) & (df['status'].shift(1) != 1)
    매도조건 = (df['status'] == 0) & (df['status'].shift(1) == 1)
    
    #누적 수익률 계산   --> 전체를 넣었다 뺏다하는데, 불려졌으면 그대로 하는 방식(복리)
    수익률 = df.loc[매도조건,'종가'].reset_index(drop=True)/df.loc[매수조건,'종가'].reset_index(drop=True)
    수익률 = 수익률 - 0.002
    
    return 수익률.cumprod().iloc[-1]
    

In [92]:
단일_이평선_수익률(df,3)

1.2037220606995058

In [93]:
# N일 이평선 중 어떤 N이 가장 효과적인가

index = range(2,61)
result = [단일_이평선_수익률(df,x) for x in index]

In [94]:
from pandas import Series

s = Series(result,index) #array를 Series 객체로 변환하면
s.sort_values(ascending = False).head() #이렇게 정렬 메서드를 사용할 수 있다.

7     1.264970
34    1.264832
33    1.264832
31    1.254641
32    1.253811
dtype: float64

### 장단기 이평선 교차(골든크로스,데드크로스)<br>
단기가 장기를 추월하는 골든크로스 때 매수 <br>
장기가 단기를 추월하는 데드크로스 때 매도 <br>

In [95]:
df1 = df.drop(labels=['ma','status'],axis=1)
df1

코스피,종가,shift
날짜,,
2020-01-02,2175.17,NaN
2020-01-03,2176.46,0.0
2020-01-06,2155.07,0.0
2020-01-07,2175.54,0.0
2020-01-08,2151.31,1.0
...,...,...
2020-08-25,2366.73,1.0
2020-08-26,2369.32,1.0
2020-08-27,2344.45,1.0


In [96]:

#sma short moving average
#lma long moving average

df1['sma'] = df1['종가'].rolling( 3 ).mean().shift(1) #오늘을 포함하지 않게 shift 1
df1['lma'] = df1['종가'].rolling( 70 ).mean().shift(1)
df1.tail()

코스피,종가,shift,sma,lma
날짜,,,,
2020-08-25,2366.73,1.0,2302.880000,2181.380857
2020-08-26,2369.32,1.0,2333.716667,2187.518286
2020-08-27,2344.45,1.0,2355.293333,2193.071286
2020-08-28,2353.80,0.0,2360.166667,2198.140000
2020-08-31,2326.17,0.0,2355.856667,2203.218429


In [107]:
cond  = df1['sma'] > df1['lma']    
df1['status'] = np.where(cond,1,0)
df1.iloc[-1,-1] = 0   #마지막 날에는 무조건 매도하는 걸로
df1

코스피,종가,shift,sma,lma,status
날짜,,,,,
2020-01-02,2175.17,NaN,NaN,NaN,0
2020-01-03,2176.46,0.0,NaN,NaN,0
2020-01-06,2155.07,0.0,NaN,NaN,0
2020-01-07,2175.54,0.0,2168.900000,NaN,0
2020-01-08,2151.31,1.0,2169.023333,NaN,0
...,...,...,...,...,...
2020-08-25,2366.73,1.0,2302.880000,2181.380857,1
2020-08-26,2369.32,1.0,2333.716667,2187.518286,1
2020-08-27,2344.45,1.0,2355.293333,2193.071286,1


In [108]:
    #bool 조건식화 하기
    매수조건 = (df1['status'] == 1) & (df1['status'].shift(1) != 1)
    매도조건 = (df1['status'] == 0) & (df1['status'].shift(1) == 1)

In [109]:
    수익률 = df1.loc[매도조건,'종가'].reset_index(drop=True)/df.loc[매수조건,'종가'].reset_index(drop=True)
    수익률 = 수익률 - 0.002
    
    수익률.cumprod().iloc[-1]

1.1978927708848046

In [105]:
def 장기_단기_이평선_수익률(df,s,l):
    df = df[['종가']].copy()
    
    #sma short moving average
    #lma long moving average

    df['sma'] = df['종가'].rolling( s ).mean().shift(1) #오늘을 포함하지 않게 shift 1
    df['lma'] = df['종가'].rolling( l ).mean().shift(1)
    df.tail()
    
    cond  = df['sma'] > df['lma']    
    df['status'] = np.where(cond,1,0)
    df.iloc[-1,-1] = 0   #마지막 날에는 무조건 매도하는 걸로
    
    #bool 조건식화 하기
    매수조건 = (df['status'] == 1) & (df['status'].shift(1) != 1)
    매도조건 = (df['status'] == 0) & (df['status'].shift(1) == 1)
    
    #누적 수익률 계산   --> 전체를 넣었다 뺏다하는데, 불려졌으면 그대로 하는 방식(복리)
    수익률 = df.loc[매도조건,'종가'].reset_index(drop=True)/df.loc[매수조건,'종가'].reset_index(drop=True)
    수익률 = 수익률 - 0.002
    
    return 수익률.cumprod().iloc[-1]


In [106]:
df1
장기_단기_이평선_수익률(df1,3,70)

1.1978927708848046